In [66]:
import os
import json
import pandas as pd

# تابع تبدیل اعداد فارسی به انگلیسی
def convert_farsi_to_english_numbers(s):
    """تبدیل اعداد فارسی به اعداد انگلیسی"""
    numbers = {
        '۰': '0', '۱': '1', '۲': '2', '۳': '3', '۴': '4',
        '۵': '5', '۶': '6', '۷': '7', '۸': '8', '۹': '9'
    }
    if isinstance(s, str):
        for persian_digit, english_digit in numbers.items():
            s = s.replace(persian_digit, english_digit)
    return s

# مشخص کردن نام فایل‌های JSON
pc_data = ['apple.json', 'acer.json', 'asus.json', 'hp.json', 'lenovo.json', 'msi.json']
mobile_data = ['honor.json', 'huawei.json', 'nokia.json', 'samsung.json', 'xiaomi.json']

all_pc_data = []
all_mobile_data = []

# خواندن داده‌های کامپیوتر
for filename in os.listdir():
    if filename in pc_data:  
        with open(filename, 'r', encoding='utf-8') as file:
            data1 = json.load(file)
            all_pc_data.append(data1) if isinstance(data1, dict) else all_pc_data.extend(data1)

# خواندن داده‌های موبایل
for filename in os.listdir():
    if filename in mobile_data:  
        with open(filename, 'r', encoding='utf-8') as file:
            data2 = json.load(file)
            all_mobile_data.append(data2) if isinstance(data2, dict) else all_mobile_data.extend(data2)

# تبدیل داده‌ها به DataFrame
df_mobile = pd.DataFrame(all_mobile_data)
df_pc = pd.DataFrame(all_pc_data)

# تغییر نام ستون‌ها برای موبایل و کامپیوتر
columns = ['id', 'link', 'image', 'name', 'price', 'rate', 'freeSend', 'status']
if not df_mobile.empty:
    df_mobile.columns = columns
if not df_pc.empty:
    df_pc.columns = columns

# پردازش داده‌های قیمت‌ها و نرخ‌ها
def process_columns(df):
    """پردازش ستون قیمت و نرخ"""
    # تبدیل اعداد فارسی
    df['price'] = df['price'].apply(lambda x: convert_farsi_to_english_numbers(x))
    df['rate'] = df['rate'].apply(lambda x: convert_farsi_to_english_numbers(x))
    
    # حذف درصد و کاما از قیمت
    df['price'] = df['price'].str.replace(r'[٪,]', '', regex=True)  
    df['price'] = pd.to_numeric(df['price'], errors='coerce')  # تبدیل به نوع عددی

    # تبدیل نرخ به عدد
    df['rate'] = pd.to_numeric(df['rate'], errors='coerce')  # تبدیل به نوع عددی
    return df

# پردازش برای کامپیوتر
if not df_pc.empty:
    df_pc = process_columns(df_pc)
    print("PC Prices and Rates after processing:")
    print(df_pc[['price', 'rate']].head(5))  # نمایش ۵ قیمت و نرخ اول برای PC

# پردازش برای موبایل
if not df_mobile.empty:
    df_mobile = process_columns(df_mobile)
    print("Mobile Prices and Rates after processing:")
    print(df_mobile[['price', 'rate']].head(5))  # نمایش ۵ قیمت و نرخ اول برای موبایل


PC Prices and Rates after processing:
   price  rate
0    4.8  True
1    4.7  True
2    NaN  True
3    NaN  True
4    5.0  True
Mobile Prices and Rates after processing:
         price  rate
0  213999000.0   4.4
1   16499000.0   4.5
2    8875000.0   4.5
3    5979000.0   4.2
4    8300000.0   4.4


In [68]:
print(combined_df1['rate'].unique())  # نمایش مقادیر یکتا در ستون rate برای کامپیوتر
print(combined_df2['rate'].unique())  # نمایش مقادیر یکتا در ستون rate برای موبایل


[1.]
[4.4 4.5 4.2 4.1 4.3 5.  3.6 4.6 3.8 4.  3.9 4.8 nan 3.  4.7 3.5 3.4 3.7
 2.8 3.1 2.  2.9 2.6 3.3 2.5 2.3 3.2 4.9 2.4 1.5 1.6]


In [70]:
import pandas as pd
import numpy as np

def clean_and_convert_rate(s):
    """پاکسازی و تبدیل مقادیر rate به عدد"""
    # ابتدا تبدیل اعداد فارسی به انگلیسی
    s = convert_farsi_to_english_numbers(s)
    
    # حذف هر چیزی که عدد نیست
    if isinstance(s, str):
        # فقط اعداد و اعشار را نگه می‌دارد
        s = ''.join(filter(lambda x: x.isdigit() or x == '.', s))
    
    # اگر رشته خالی ماند، None را برمی‌گرداند
    return float(s) if s else np.nan

# پردازش ستون rate برای DataFrame کامپیوتر
combined_df1['rate'] = combined_df1['rate'].apply(clean_and_convert_rate)

# پردازش ستون rate برای DataFrame موبایل
combined_df2['rate'] = combined_df2['rate'].apply(clean_and_convert_rate)

# چاپ مقادیر جدید برای بررسی
print(combined_df1['rate'].head(5))
print(combined_df2['rate'].head(5))


0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: rate, dtype: float64
0    4.4
1    4.5
2    4.5
3    4.2
4    4.4
Name: rate, dtype: float64


Missing value

In [74]:
# تعداد مقادیر گمشده در DataFrame کامپیوتر
missing_pc_price = df_pc['price'].isnull().sum()
missing_pc_rate = df_pc['rate'].isnull().sum()

print(f"Number of missing values in PC Price: {missing_pc_price}")
print(f"Number of missing values in PC Rate: {missing_pc_rate}")

# تعداد مقادیر گمشده در DataFrame موبایل
missing_mobile_price = df_mobile['price'].isnull().sum()
missing_mobile_rate = df_mobile['rate'].isnull().sum()

print(f"Number of missing values in Mobile Price: {missing_mobile_price}")
print(f"Number of missing values in Mobile Rate: {missing_mobile_rate}")


Number of missing values in PC Price: 442
Number of missing values in PC Rate: 0
Number of missing values in Mobile Price: 684
Number of missing values in Mobile Rate: 63


In [97]:
import pandas as pd
import numpy as np

# فرض کنید df_pc و df_mobile از قبل تعریف شده‌اند

# تابعی برای پر کردن مقادیر گمشده با استفاده از نام
def fill_missing_values(df):
    # اطمینان از اینکه همه مقادیر در ستون name از نوع رشته هستند
    df['name'] = df['name'].astype(str)

    for index, row in df.iterrows():
        # اگر قیمت گمشده است
        if pd.isnull(row['price']):
            # پیدا کردن دستگاه‌های مشابه بر اساس نام
            similar_items = df[df['name'].str.contains(row['name'], na=False, case=False)]
            if not similar_items.empty:
                df.at[index, 'price'] = similar_items['price'].mean()  # میانگین قیمت دستگاه‌های مشابه

        # اگر امتیاز گمشده است
        if pd.isnull(row['rate']):
            # پیدا کردن دستگاه‌های مشابه بر اساس نام
            similar_items = df[df['name'].str.contains(row['name'], na=False, case=False)]
            if not similar_items.empty:
                df.at[index, 'rate'] = similar_items['rate'].mean()  # میانگین امتیاز دستگاه‌های مشابه

    return df

# پر کردن مقادیر گمشده برای کامپیوتر
df_pc = fill_missing_values(df_pc)

# پر کردن مقادیر گمشده برای موبایل
df_mobile = fill_missing_values(df_mobile)

# نمایش نتایج
print("PC Data after filling missing values:")
print(df_pc[['name', 'price', 'rate']].head(5))

print("\nMobile Data after filling missing values:")
print(df_mobile[['name', 'price', 'rate']].head(5))


C:\Users\User\AppData\Local\Temp\ipykernel_16284\193485577.py:15: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  similar_items = df[df['name'].str.contains(row['name'], na=False, case=False)]


PC Data after filling missing values:
            name  price  rate
0   ۷٪۲۳,۷۹۰,۰۰۰    4.8  True
1  ۱۹٪۳۱,۸۹۰,۰۰۰    4.7  True
2  ۱۲٪۲۸,۹۹۹,۰۰۰    4.0  True
3  ۱۲٪۲۰,۹۹۹,۵۵۰    NaN  True
4  ۱۳٪۲۱,۰۹۹,۰۰۰    5.0  True

Mobile Data after filling missing values:
                                                name        price  rate
0  گوشی موبایل آنر مدل X9a دو سیم کارت ظرفیت 256 ...  213999000.0   4.4
1  گوشی موبایل آنر مدل 70 دو سیم کارت ظرفیت 128 گ...   16499000.0   4.5
2  گوشی موبایل آنر مدل X8a دو سیم کارت ظرفیت 128 ...    8875000.0   4.5
3  گوشی موبایل آنر مدل  X7a دو سیم کارت ظرفیت 128...    5979000.0   4.2
4  گوشی موبایل آنر مدل X8 دو سیم کارت ظرفیت 128گی...    8300000.0   4.4
